# Afficher la campagne et les interactions

Dans le premier bloc-notes `Personalize_BuildCampaign.ipynb`, vous avez créé et déployé correctement un modèle de recommandation utilisant le deep learning avec Amazon Personalize.

Ce bloc-notes étend ce modèle et vous guide pour vous permettre de réagir au comportement en temps réel des utilisateurs. Si leur intention change pendant qu'ils regardent un film, vous verrez des recommandations modifiées en fonction de ce comportement.

Il présentera également un code de démonstration permettant de simuler le comportement de l'utilisateur qui sélectionne des films avant que les recommandations soient renvoyées.

Ci-dessous, nous commençons par importer les bibliothèques dont nous avons besoin pour interagir avec Personalize.

In [ ]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import uuid

La ligne ci-dessous récupère les variables partagées du premier bloc-notes.

In [ ]:
%store -r

In [ ]:
# Setup and Config
# Recommendations from Event data
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's Event Streaming
personalize_events = boto3.client(service_name='personalize-events')

## Création d'un outil de suivi des événements

Pour que votre système de recommandation puisse répondre à des événements en temps réel, vous avez besoin d'un outil de suivi des événements. Le code ci-dessous en génère un et peut être utilisé pour la suite de cet atelier. N'hésitez pas à remplacer son nom par un nom plus descriptif.

In [ ]:
response = personalize.create_event_tracker(
    name='MovieClickTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']

In [ ]:
event_tracker_arn = response['eventTrackerArn']

## Configuration des données source

Ci-dessus figure votre identifiant de suivi qui a été affecté à une variable. Vous n'avez donc rien à faire. Les lignes ci-dessous permettent de configurer les données utilisées pour les recommandations, afin que vous puissiez afficher la liste de films plus tard.

In [ ]:
# First load items into memory
items = pd.read_csv('./ml-100k/u.item', sep='|', usecols=[0,1], encoding='latin-1', names=['ITEM_ID', 'TITLE'], index_col='ITEM_ID')

def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)-1
    return items.loc[movie_id]['TITLE']

## Obtention de recommandations

Tout d'abord, nous allons afficher à nouveau les recommandations du bloc-notes précédent :

In [ ]:
recommendations_df

## Simulation du comportement des utilisateurs

Les lignes ci-dessous fournissent un exemple de code qui simule un utilisateur interagissant avec un article donné. Vous obtiendrez alors des recommandations différentes de celles de départ.

In [ ]:
session_dict = {}

In [ ]:
def send_movie_click(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[USER_ID]
    except:
        session_dict[USER_ID] = str(uuid.uuid1())
        session_ID = session_dict[USER_ID]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= USER_ID,
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': 'EVENT_TYPE',
        'properties': event_json
        }]
)

Immédiatement en dessous de cette ligne, l'outil de suivi sera mis à jour comme si l'utilisateur avait cliqué sur un titre.


Si la table générée par les cellules ci-dessous ne modifie pas les recommandations, essayez simplement un autre nombre aléatoire de 3 chiffres dans la cellule ci-dessus et exécutez à nouveau les deux cellules. Vous constaterez qu'une troisième colonne de recommandations a été générée.

In [ ]:
# Pick a movie, we will use ID 270 or Gattaca
movie_to_click = 270
movie_title_clicked = get_movie_title(movie_to_click)
send_movie_click(USER_ID=str(user_id), ITEM_ID=movie_to_click)

Après avoir exécuté ce bloc, vous verrez les modifications dans les recommandations maintenant que le suivi des événements est activé et que vous avez envoyé les événements au service.

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    title = get_movie_title(item['itemId'])
    recommendation_list.append(title)
    
new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_title_clicked])

recommendations_df = recommendations_df.join(new_rec_DF)
recommendations_df

## Conclusion

Vous pouvez voir maintenant que les recommandations sont modifiées en changeant le film avec lequel un utilisateur interagit. Ce système peut être modifié pour toute application où les utilisateurs interagissent avec une collection d'articles. Ces outils sont disponibles à tout moment pour commencer à explorer ce qui est possible avec les données dont vous disposez.

Exécutez la cellule ci-dessous pour stocker les valeurs nécessaires au bloc-notes de nettoyage.

Enfin, lorsque vous êtes prêt à supprimer les articles de votre compte, ouvrez le bloc-notes `Cleanup.ipynb` et exécutez les étapes qui s'y trouvent.


In [ ]:
%store event_tracker_arn